In [1]:
import pandas as pd

In [60]:
def calculate_cpr(term_months, prepayment_multiplier=1.0, seasoning=0, psa_1=0.002, psa_30=0.06):
    """
    Calculate the monthly Conditional Prepayment Rate (CPR) for a loan.

    Args:
        term_months (int): Term of the loan in months.
        prepayment_multiplier (float): Multiplier for prepayment (default 1.0, i.e., 100%).
        seasoning (int): Number of months seasoned (0-12).
        psa_1 (float): PSA CPR at month 1 (default 0.2% = 0.002).
        psa_30 (float): PSA CPR at month 30 (default 6% = 0.06).

    Returns:
        list: Monthly CPRs for each month from 1 to term_months.
    """
    psa_1 = psa_1*prepayment_multiplier
    psa_30 = psa_30*prepayment_multiplier
    cpr_list = []
    cpr_increment = psa_1 # Linear growth from month 1 to 30

    # Adjust starting CPR for seasoning
    if seasoning == 0:
        starting_cpr = psa_1
    else:
        starting_cpr = psa_1*seasoning

    for t in range(2, term_months + 2):
        if t <= 30-seasoning:
            cpr = starting_cpr + (t - 1) * cpr_increment
            cpr = min(cpr, psa_30)
        else:
            cpr = psa_30
        cpr_list.append(cpr)

    return cpr_list

In [61]:
def calculate_initial_monthly_payment(balance, annual_rate, term_months, seasoning_months=0):
    """
    Calculate the initial monthly payment for a mortgage.

    Args:
        balance (float): Remaining mortgage balance.
        annual_rate (float): Annual interest rate (e.g., 0.05 for 5%).
        term_months (int): Original term of the loan in months.
        seasoning_months (int): Number of months seasoned (payments already made).

    Returns:
        float: Initial monthly payment at the given seasoning.
    """
    # Remaining term after seasoning
    remaining_term = term_months - seasoning_months
    if remaining_term <= 0:
        raise ValueError("Seasoning months must be less than term of loan.")

    monthly_rate = annual_rate / 12
    if monthly_rate == 0:
        return balance / remaining_term

    payment = balance * (monthly_rate * (1 + monthly_rate) ** remaining_term) / ((1 + monthly_rate) ** remaining_term - 1)
    return payment

In [62]:
import pandas as pd

def cpr_smm_dataframe(balance, mortgage_rate, passthrough_rate, term_months, prepayment_multiplier=1.0, seasoning=0, psa_1=0.002, psa_30=0.06):
    """
    Generate a DataFrame with CPR and SMM values for each month.

    Args:
        balance (float): Remaining mortgage balance.
        mortgage_rate (float): Annual mortgage rate (e.g., 0.05 for 5%).
        term_months (int): Term of the loan in months.
        prepayment_multiplier (float): Multiplier for prepayment (default 1.0).
        seasoning (int): Number of months seasoned (0-12).
        psa_1 (float): PSA CPR at month 1.
        psa_30 (float): PSA CPR at month 30.

    Returns:
        pd.DataFrame: DataFrame with columns 'CPR' and 'SMM'.
    """
    # Use the existing cpr_list variable, which is already calculated

    cpr_list = calculate_cpr(term_months, prepayment_multiplier, seasoning, psa_1, psa_30)
    df = pd.DataFrame({
        'CPR': cpr_list,
        'SMM': [1 - (1 - cpr) ** (1 / 12) for cpr in cpr_list]
        }, index=range(1, len(cpr_list) + 1))
    df['Start Month. Bal.'] = None
    df.at[1, 'Start Month. Bal.'] = balance
    df['Month. Paym.'] = None
    df.at[1, 'Month. Paym.'] = calculate_initial_monthly_payment(balance, mortgage_rate, term_months, seasoning)
    df["Int by Mort Hold"] = None
    df.at[1, "Int by Mort Hold"] = df.at[1, 'Start Month. Bal.'] * (mortgage_rate / 12)
    df["Int to P-T Inv"] = None
    df.at[1, "Int to P-T Inv"] = df.at[1, 'Start Month. Bal.'] * (passthrough_rate / 12)
    df["Scheduled Princ."] = None
    df.at[1, "Scheduled Princ."] = df.at[1, 'Month. Paym.'] - df.at[1, "Int by Mort Hold"]
    df["Prepaym."] = None
    df.at[1, "Prepaym."] = (df.at[1, "Start Month. Bal."]- df.at[1, "Scheduled Princ."])*(1-(1-df.at[1, 'CPR'])**(1/12))
    df["Total Princ."] = None
    df.at[1, "Total Princ."] = df.at[1, "Scheduled Princ."] + df.at[1, "Prepaym."]
    df["End Month. Bal."] = None
    df.at[1, "End Month. Bal."] = df.at[1, 'Start Month. Bal.'] - df.at[1, "Total Princ."]

    for i in range(2, len(df) + 1):
        if df.at[i-1, "End Month. Bal."] is not None and df.at[i-1, "End Month. Bal."] > 0:
            df.at[i, "Start Month. Bal."] = df.at[i-1, "End Month. Bal."]
            df.at[i, "Month. Paym."] = df.at[i-1, "End Month. Bal."] * (mortgage_rate / 12) / (1-(1+ mortgage_rate/12)**(-(term_months-seasoning-i+1)))
            df.at[i, "Int by Mort Hold"] = mortgage_rate* df.at[i-1, "End Month. Bal."] / 12
            df.at[i, "Int to P-T Inv"] = df.at[i, "Int by Mort Hold"] * (passthrough_rate / mortgage_rate)
            df.at[i, "Scheduled Princ."] = df.at[i, "Month. Paym."] - df.at[i, "Int by Mort Hold"]
            df.at[i, "Prepaym."] = (df.at[i, "Start Month. Bal."]- df.at[i, "Scheduled Princ."])*(1-(1-df.at[i, 'CPR'])**(1/12))
            df.at[i, "Total Princ."] = df.at[i, "Scheduled Princ."] + df.at[i, "Prepaym."]
            df.at[i, "End Month. Bal."] = df.at[i, "Start Month. Bal."] - df.at[i, "Total Princ."]
    #cols_to_format = [col for col in df.columns if col not in ['CPR', 'SMM']]
    #df[cols_to_format] = df[cols_to_format].applymap(lambda x: f"{float(x):.2f}" if x not in [None, 'None'] else x)
    
    print("Sum of monthly payments:", df["Month. Paym."].apply(pd.to_numeric, errors='coerce').sum())
    print("Sum of interest paid by mortgage holders:", df["Int by Mort Hold"].apply(pd.to_numeric, errors='coerce').sum())
    print("Sum of interest paid to pass-through investor:", df["Int to P-T Inv"].apply(pd.to_numeric, errors='coerce').sum())
    print("Interest earned by MBS issuer:", df["Int by Mort Hold"].apply(pd.to_numeric, errors='coerce').sum() - df["Int to P-T Inv"].apply(pd.to_numeric, errors='coerce').sum())
    print("Sum of prepayments:", df["Prepaym."].apply(pd.to_numeric, errors='coerce').sum())
    print("Sum of total principal payments:", df["Total Princ."].apply(pd.to_numeric, errors='coerce').sum())

    return df

In [63]:
df = cpr_smm_dataframe(400000, 0.06,0.05, 240, prepayment_multiplier=1.5, seasoning=2)
df

Sum of monthly payments: 341147.0254882459
Sum of interest paid by mortgage holders: 174326.24630917484
Sum of interest paid to pass-through investor: 145271.87192431238
Interest earned by MBS issuer: 29054.374384862458
Sum of prepayments: 233179.22082092907
Sum of total principal payments: 400000.0000000001


,CPR,SMM,Start Month. Bal.,Month. Paym.,Int by Mort Hold,Int to P-T Inv,Scheduled Princ.,Prepaym.,Total Princ.,End Month. Bal.
1,0.009,0.000753,400000,2878.214079,2000.0,1666.666667,878.214079,300.583269,1178.797348,398821.202652
2,0.012,0.001006,398821.202652,2876.046462,1994.106013,1661.755011,881.940449,400.144856,1282.085305,397539.117346
3,0.015,0.001259,397539.117346,2873.154475,1987.695587,1656.412989,885.458889,499.258844,1384.717733,396154.399613
4,0.018,0.001513,396154.399613,2869.538102,1980.771998,1650.643332,888.766104,597.846863,1486.612966,394667.786647
5,0.021,0.001767,394667.786647,2865.19787,1973.338933,1644.449111,891.858937,695.83091,1587.689848,393080.096799
...,...,...,...,...,...,...,...,...,...,...
236,0.090,0.007828,1491.438178,502.125785,7.457191,6.214326,494.668594,7.803131,502.471725,988.966452
237,0.090,0.007828,988.966452,498.194933,4.944832,4.120694,493.250101,3.880676,497.130777,491.835675
238,0.090,0.007828,491.835675,494.294854,2.459178,2.049315,491.835675,-0.0,491.835675,-0.0
239,0.090,0.007828,None,None,None,None,None,None,None,None


In [44]:
type(df.at[1,'Start Month. Bal.'])  


int